In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from tqdm import tqdm

# Parameters for data generation
sh = int(1e4)  # Number of samples

# Define low and high bounds for each dimension
lows = [0, 0, 1.0, 1.0, 0.1]  # [t, phi0, omega0, Mc, eta]
highs = [5, 2 * np.pi, 10.0, 2.0, 0.4]

# Sample uniformly from the 5D space
samples = np.random.uniform(low=lows, high=highs, size=(sh, 5))
results = np.zeros((sh, 2))  # To store [omega(t), phi(t)]

# Define the true system
def true_system(t, y, Mc, eta):
    omega, phi = y
    d_omega = -eta * omega + (1 / Mc) * np.sin(phi)
    d_phi = omega
    return [d_omega, d_phi]
def true_system_torch(omega, phi, Mc, eta):
    d_omega = -eta * omega + (1 / Mc) * torch.sin(phi)
    d_phi = omega
    return d_omega, d_phi
# Solve the system up to t
def get_PHI_omg(t0, phi0, omg0, Mc, eta, t_epoch):
    sol = solve_ivp(
        true_system,
        [t0, t_epoch],
        y0=[omg0, phi0],
        args=(Mc, eta),
        t_eval=[t_epoch],
        rtol=1e-9,
        atol=1e-9
    )
    return sol.y[1, 0], sol.y[0, 0]  # phi(t), omega(t)

In [9]:
# Loop over all samples and compute the results
for i in tqdm(range(sh), desc="Generating data"):
    t, phi0_, omg0_, Mc_, eta_ = samples[i]
    phi_val, omg_val = get_PHI_omg(t0=0, phi0=phi0_, omg0=omg0_, Mc=Mc_, eta=eta_, t_epoch=t)
    results[i, 0] = omg_val
    results[i, 1] = phi_val

# Reorder input: [t, Mc, eta, omega0, phi0]
X_data = samples[:, [0, 3, 4, 2, 1]]  # [t, Mc, eta, omega0, phi0]
Y_data = results  # [omega(t), phi(t)]

# Convert to torch tensors (float32)
X_data = torch.tensor(X_data, dtype=torch.float32)
Y_data = torch.tensor(Y_data, dtype=torch.float32)

Generating data: 100%|███████████████████| 10000/10000 [00:16<00:00, 595.07it/s]


In [10]:
data_tensor = torch.cat([X_data, Y_data], dim=1)
torch.save(data_tensor, "./data_test.pt")

In [11]:
# Neural Network Model (PINN)
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)  # outputs omega and phi
        )

    def forward(self, x):
        return self.net(x)

In [15]:
def physics_loss(model, X_batch):
    X_batch = X_batch.clone().detach().requires_grad_(True)

    t = X_batch[:, 0:1]
    Mc = X_batch[:, 1:2]
    eta = X_batch[:, 2:3]

    y_pred = model(X_batch)
    omega_pred = y_pred[:, 0:1]
    phi_pred = y_pred[:, 1:2]

    d_omega_pred = torch.autograd.grad(
        outputs=omega_pred,
        inputs=X_batch,
        grad_outputs=torch.ones_like(omega_pred),
        create_graph=True
    )[0][:, 0:1]  # ∂ω/∂t

    d_phi_pred = torch.autograd.grad(
        outputs=phi_pred,
        inputs=X_batch,
        grad_outputs=torch.ones_like(phi_pred),
        create_graph=True
    )[0][:, 0:1]  # ∂φ/∂t

    # Calculate true dynamics using Mc and eta
    d_omega_true, d_phi_true = true_system_torch(omega_pred, phi_pred, Mc, eta)

    loss = ((d_omega_pred - d_omega_true) ** 2).mean() + ((d_phi_pred - d_phi_true) ** 2).mean()
    return loss


In [16]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Load the generated data
dataset = TensorDataset(X_data, Y_data)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the model and optimizer
model = PINN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Using device: mps


In [ ]:
# Training loop
epochs = 200
for epoch in range(epochs):
    total_loss = 0
    for X_batch, Y_batch in loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        
        optimizer.zero_grad()
        loss = physics_loss(model, X_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    if epoch % 2 == 0:
        print(f"Epoch {epoch}: loss = {total_loss / len(loader):.6f}")

Epoch 0: loss = 0.001286
Epoch 2: loss = 0.000023
Epoch 4: loss = 0.000014
